Be sure to choose a GPU instance to run the full size model.

In [1]:
!nvidia-smi

Sat Jul 23 06:23:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Colab already has JAX and PyTorch (used for the dataloading) preinstalled, so we only need to install a few additional packages. 

In [2]:
!pip install datasets -q
!pip install flax -q
!pip install wandb -q
!git clone 'https://github.com/lindermanlab/S5.git'

     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 212 kB 57.0 MB/s 
     |████████████████████████████████| 1.1 MB 2.4 MB/s 
     |████████████████████████████████| 140 kB 62.2 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 127 kB 46.0 MB/s 
     |████████████████████████████████| 271 kB 64.8 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 65.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 197 kB 4.7 MB/s 
     |████████████████████████████████| 145 kB 57.5 MB/s 
     |████████████████████████████████| 21

In [3]:
HOME_DIR = '/content/S5/src'

import sys
import os
sys.path.append(os.path.join(HOME_DIR))

import train
import dataloading

Here we create and cache the IMDB dataset. This will create a directory structure and add the train and validation files.

In [4]:
dataloading.create_and_cache_imdb_dataset()

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/25000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/25000 [00:00<?, ?ex/s]

  0%|          | 0/25000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

Uncomment and run the following cell if you would like to log this run to Weights and Biases. 

In [ ]:
# !wandb login

Here we train the model. Be sure to change the first 3 variables if you would like to log to wandb.

In [5]:
train.train(
        USE_WANDB=False,  # log with wandb
        wandb_project=None,  # wandb project name
        wandb_entity=None,  # wandb entity name
        dir_name="./S5/data",  # data directory
        dataset="imdb-classification",
        n_layers=4,  # Number of layers of network
        d_model=64,  # Number of features, i.e. H, dimension of layer inputs/outputs
        ssm_size=64,  # SSM Latent size, i.e. P
        blocks=1,  # How many blocks to initialize A with, see Appendix discussion
        BC_init="dense",  # B, C init approach, [dense, dense_columns, factorized]
        k=-1,  # rank to use for low-rank factorization
        discretization="zoh",  # options: [zoh, bilinear]
        mode="pool",  # use mean pooling or just take last element of state sequence
        prenorm=True,  # True: use prenorm, False: use postnorm
        batchnorm=False,  # True: use batchnorm, False: no batchnorm
        bn_momentum=-1,  # batchnorm momentum if used
        bsz=50,  # batch size
        epochs=40,
        early_stop_patience=15,
        ssm_lr=0.0095,  # learning rate to use for SSM parameters
        lr_factor=3,  # learning rate of non-SSM parameters is lr_factor*ssm_lr
        cosine_anneal=False,  # True: use cosine annealing, False: no annealing
        warmup_end=0,  # Epoch to end linear warmup on; 0 means no warmup
        lr_patience=5,  # patience to use for decay learning rate on plateau
        reduce_factor=0.2,  # decay rate for decay learning rate on plateau
        p_dropout=0.12,  # dropout rate
        weight_decay=0.008,
        opt_config="noBCdecay",  # B and/or C weight decay Options: [noBdecay, BandCdecay, noBCdecay]
        jax_seed=8915814  # set JAX randomness, affects B,C,D,Delta init and dropout
        )

[*] Setting Randomness...
Using Vocab train and val steps because dataset is: imdb-classification
[*] Generating LRA-text (IMDB) Classification Dataset
[*] Starting S5 Training on `imdb-classification` =>> Initializing...
configuring optimization with no weight decay on B or C
[*] Starting Training Epoch 1...
using constant lr for epoch 1


100%|██████████| 500/500 [00:52<00:00,  9.58it/s]


[*] Running Epoch 1 Test...


100%|██████████| 500/500 [00:19<00:00, 25.29it/s]



=>> Epoch 1 Metrics ===
	Train Loss: 0.68338  --Test Loss: 0.65591 -- Test Accuracy: 0.6198
	Best Val Loss: 0.65591 -- Best Val Accuracy: 0.6198 at Epoch 1
	Best Test Loss: 0.65591 -- Best Test Accuracy: 0.6198 at Epoch 1

[*] Starting Training Epoch 2...
using constant lr for epoch 2


...


=>> Epoch 39 Metrics ===
	Train Loss: 0.16784  --Test Loss: 0.39615 -- Test Accuracy: 0.8614
	Best Val Loss: 0.36679 -- Best Val Accuracy: 0.8651 at Epoch 37
	Best Test Loss: 0.36679 -- Best Test Accuracy: 0.8651 at Epoch 37

[*] Starting Training Epoch 40...
using constant lr for epoch 40


100%|██████████| 500/500 [00:26<00:00, 18.84it/s]


[*] Running Epoch 40 Test...


100%|██████████| 500/500 [00:09<00:00, 54.53it/s]



=>> Epoch 40 Metrics ===
	Train Loss: 0.15689  --Test Loss: 0.41700 -- Test Accuracy: 0.8614
	Best Val Loss: 0.36679 -- Best Val Accuracy: 0.8651 at Epoch 37
	Best Test Loss: 0.36679 -- Best Test Accuracy: 0.8651 at Epoch 37

